<a href="https://colab.research.google.com/github/Sirfowahid/FFN/blob/master/FNN_DataNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

In [2]:
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

labels = data[:,0]
data   = data[:,1:]

In [3]:

dataT   = torch.tensor( data ).float()
labelsT = torch.tensor( labels ).long()

train_data,test_data, train_labels,test_labels = train_test_split(dataT, labelsT, test_size=.1)

train_data = train_data/torch.max(train_data)
test_data  = test_data/torch.max(test_data)


train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

batchsize    = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [4]:

print('Training data range %g to %g' 
      %(torch.min(train_loader.dataset.tensors[0]),torch.max(train_loader.dataset.tensors[0])) )

print('Test data range %g to %g' 
      %(torch.min(test_loader.dataset.tensors[0]),torch.max(test_loader.dataset.tensors[0])) )

Training data range 0 to 1
Test data range 0 to 1


In [5]:
def createTheMNISTNet():

  class mnistNet(nn.Module):
    def __init__(self):
      super().__init__()

      
      self.input = nn.Linear(784,64)
      
      
      self.fc1 = nn.Linear(64,32)
      self.fc2 = nn.Linear(32,32)

      
      self.output = nn.Linear(32,10)

    
    def forward(self,x):
      x = F.relu( self.input(x) )
      x = F.relu( self.fc1(x) )
      x = F.relu( self.fc2(x) )
      return self.output(x)
  
  
  net = mnistNet()
  
  
  lossfun = nn.CrossEntropyLoss()

  
  optimizer = torch.optim.SGD(net.parameters(),lr=.01)

  return net,lossfun,optimizer

In [6]:
def funtion2trainTheModel():

  
  numepochs = 60
  
  
  net,lossfun,optimizer = createTheMNISTNet()

  
  losses    = torch.zeros((numepochs,2))
  trainAcc  = []
  testAcc   = []


  
  for epochi in range(numepochs):

    
    batchAcc  = []
    batchLoss = []
    for X,y in train_loader:

      
      yHat = net(X)
      loss = lossfun(yHat,y)

      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      
      batchLoss.append(loss.item())

      
      matches = torch.argmax(yHat,axis=1) == y     
      matchesNumeric = matches.float()             
      accuracyPct = 100*torch.mean(matchesNumeric) 
      batchAcc.append( accuracyPct )               
    trainAcc.append( np.mean(batchAcc) )

    
    losses[epochi,0] = np.mean(batchLoss)

    
    X,y = next(iter(test_loader)) 
    with torch.no_grad():
      yHat = net(X)
      
    
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )
    loss = lossfun(yHat,y)
    losses[epochi,1] = loss.item()

  

  
  return trainAcc,testAcc,losses,net
